# Camera calibration
Camera calibration consists of two steps: intrinsic calibration and hand-eye calibration.

The purpose of intrinsic calibration is to compute the intrinsic parameters of the camera (the focal length, the optical center)and distortion parameters (radial distortion and tangential distortion coefficient). These parameters will be used to convert the pixel coordinates to real world coordinates. A general procedure is listed in the following:


1.   Attached the camera to the end-effector of the robot arm
2.   Print an 8x8 chessboard(or something similar), each square with a known size;
3. Fix the pattern on the flat surface
4. Use OpenCV to take 15-25 images from different angles and distances
5. Process the images using OpenCV library to obtain the intrinsic and distortion parameters.

Some python script template are listed in the following for reference.



In [ ]:
# this script is used for image collection
import cv2
cap = cv2.VideoCapture(0)
i = 0
img_name_1 = "calib_img_"
while True:
    ret, frame = cap.read()
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('s'):
        i += 1
        img_name = img_name_1 + str(i) + ".jpg"
        cv2.imwrite(img_name, frame)
    elif cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# this script is used for calculation of the intrinsic and distortion parameters
import cv2
import numpy as np
import glob

# Chessboard dimensions
chessboard_size = (8, 8)
square_size = 20  # mm

# Prepare object points (0,0,0), (1,0,0) ... scaled by square size
objp = np.zeros((np.prod(chessboard_size), 3), np.float32)
objp[:, :2] = np.indices(chessboard_size).T.reshape(-1, 2)
objp *= square_size

objpoints = []
imgpoints = []

images = glob.glob('calib_img_*.jpg')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

with open("camera_calibration.pkl", "wb") as f:
    pickle.dump((K, dist), f)

print("Calibration complete. Camera matrix:\n", K)
print("Distortion coefficients:\n", dist)

The purpose of Hand-Eye Calibration is to relate what the camera sees to where the robot end effector moves. It is a process to determine the relative position and orientation of a robot-mounted camera with respect to the robot's end-effector.

A general procedure is listed in the following:
1. Move the robot to at least 10 different poses, preferably with some rotation.
2. At each pose, take a picture of the chessboard or printed pattern
3. Record the end-effector pose (transformation matrix) in base frame
4. solve hand-eye calibration

A script template is provided in the following for reference.

In [ ]:
# This step post-processes all the images captured in different end-effector poses
# By utilizing the intrinsic parameters and distortion coefficients obtained in the
# intrinsic calibration, the transformation matrix of target pose in the camera frame,
# T_target2cam(R_target2cam, t_target2cam) for each pose is obtained.
import cv2
import numpy as np

# Chessboard dimensions
chessboard_size = (8, 8)
square_size = 20  # mm

# Prepare object points (3D points in target frame)
objp = np.zeros((np.prod(checkerboard_size), 3), np.float32)
objp[:, :2] = np.mgrid[0:checkerboard_size[0], 0:checkerboard_size[1]].T.reshape(-1, 2)
objp *= square_size  # scale to real world units

# Load image and find corners
img = cv2.imread('checkerboard_image.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, checkerboard_size)

if ret:
    # Refine corner detection
    corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))

    # Camera intrinsics (from intrinsic calibration mentioned above)
    K = ...  # 3x3 intrinsic matrix
    dist_coeffs = ...  # distortion coefficients

    # Solve PnP to get target pose in camera frame
    ret, rvec, tvec = cv2.solvePnP(objp, corners, K, dist_coeffs)

    # Convert rotation vector to rotation matrix, and position vector
    R_target2cam, _ = cv2.Rodrigues(rvec)
    t_target2cam = tvec


In [ ]:
# This step estimates the transmation matrix T_cam2gripper (R_cam2gripper, t_cam2gripper),
# based on the transformation matrix T_gripper2base (R_gripper2base, t_gripper2base) and
# the transformation matrix T_target2cam(R_target2cam, t_target2cam).
import cv2
import numpy as np

# Lists of 3x3 Rotation matrix and 3x1 vectors
# both can be obtained from transformation matrix determined via
# forward kinematics
R_gripper2base = []
t_gripper2base = []
# Append all the data here...
# R_gripper2base.append(R_base_ee)
# t_gripper2base.append(t_base_ee)

R_target2cam = []     # from solvePnP
t_target2cam = []
# Append all the data here
# R_target2cam.append(R_cam_target)
# t_target2cam.append(t_cam_target)

R_cam2gripper, t_cam2gripper = cv2.calibrateHandEye(
    R_gripper2base, t_gripper2base,
    R_target2cam, t_target2cam,
    method=cv2.CALIB_HAND_EYE_TSAI
)